<a href="https://colab.research.google.com/github/MarMarhoun/freelance_work/blob/main/side_projects/NLP_projs/eda_streamlit/video_sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video summarization using reinforcement learning, GANs and streamlit and tensorflow

The file **video_sum.ipynb** contains an existing code for video summarization using reinforcement learning, GANs, Streamlit, and TensorFlow, we can add a reward system based on user feedback to improve the summarization model over time. Here is an example of how you could implement this:

First, let's import the necessary libraries:W

In [ ]:
import os
import json
import datetime
import random
import string
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from html2image import Html2Image
import youtube_transcript_api
import streamlit as st
from langchain import OpenAI
from llama_index import StorageContext, load_index_from_storage
from llama_index import VectorStoreIndex, GPTVectorStoreIndex, LLMPredictor, PromptHelper, ServiceContext
from langchain.chat_models import ChatOpenAI

Next, let's define the reward function based on user feedback:



In [ ]:
def reward_function(user_feedback, original_summary, generated_summary):
    if user_feedback == "like":
        reward = 1
    elif user_feedback == "dislike":
        reward = -1
    else:
        reward = 0

    if original_summary is not None and generated_summary is not None:
        rouge_score = nltk.translate.meteor_score.meteor_score([original_summary], [generated_summary])
        reward += (0.5 * rouge_score)

    return reward

Now, let's create a function to train the model using reinforcement learning:



In [ ]:
def train_model(episodes, max_seq_length, learning_rate, batch_size, summary_dir):
    # Load the dataset
    documents = load_documents(summary_dir)

    # Preprocess the data
    tokenizer, sequences, input_sequences, target_sequences = preprocess_data(documents, max_seq_length)

    # Define the model architecture
    inputs = Input(shape=(max_seq_length,))
    lstm = LSTM(units=256, return_sequences=True)(inputs)
    lstm = LSTM(units=256)(lstm)
    outputs = Dense(units=1, activation='linear')(lstm)

    # Compile the model
    model = Model(inputs=inputs, outputs=outputs)
    optimizer = Adam(lr=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    # Train the model
    for episode in range(episodes):
        print(f'Episode: {episode}')
        summary_pairs = generate_summary_pairs(documents)
        random.shuffle(summary_pairs)

        for i in range(0, len(summary_pairs), batch_size):
            minibatch = summary_pairs[i:i+batch_size]
            user_feedback = [random.choice(['like', 'dislike']) for _ in range(len(minibatch))]
            original_summaries = [pair[0] for pair in minibatch]
            generated_summaries = [pair[1] for pair in minibatch]

            rewards = np.array([reward_function(fb, os.path.basename(os.path.splitext(os.path.split(s)[1])[0]), gs) for fb, s, gs in zip(user_feedback, original_summaries, generated_summaries)])
            target_floats = rewards + model.predict(input_sequences[i:i+batch_size])
            model.train_on_batch(input_sequences[i:i+batch_size], target_floats)

    # Save the model
    model.save('video_summarization_model.h5')

Here is the function to generate summary pairs for training:



In [ ]:
def generate_summary_pairs(documents):
    summary_pairs = []
    for document in documents:
        # Generate a summary using the original model
        original_summary = generate_summary(document)

        # Generate a summary using the reinforcement learning model
        generated_summary = generate_summary_with_reinforcement_learning(document)

        # Add the pair to the list
        summary_pairs.append((original_summary, generated_summary))

    return summary_pairs

Now, let's create a function to generate a summary using the original model:



In [ ]:
def generate_summary(document):
    # Extract the video ID from the document
    video_id = os.path.splitext(os.path.split(document)[1])[0]

    # Load the transcript
    transcript = youtube_transcript_api.get_transcript(video_id)

    # Generate the summary
    summary = ' '.join([line['text'] for line in transcript])

    return summary

Finally, let's create a function to generate a summary using the reinforcement learning model:

In [ ]:
def generate_summary_with_reinforcement_learning(document):
    # Load the model
    model = load_model('video_summarization_model.h5')

    # Tokenize the document
    tokenizer = load_tokenizer('tokenizer.json')
    sequences = tokenizer.texts_to_sequences([document])

    # Generate the summary
    summary = ''
    for i in range(len(sequences[0])):
        input_sequence = sequences[0][:i+1]
        input_sequence = np.array(input_sequence).reshape(1, -1)

        # Predict the next word
        prediction = model.predict(input_sequence)
        next_word_index = np.argmax(prediction)

        # Decode the next word
        next_word = tokenizer.index_word[next_word_index]

        # Add the next word to the summary
        summary += next_word + ' '

    return summary

This code provides a basic implementation of reinforcement learning for video summarization. The model is trained using a reward function that takes into account user feedback and the similarity between the original and generated summaries. The model is then used to generate summaries for new videos. Note that this implementation is simplified and may not provide the best results. For a more advanced implementation, you could consider using a pre-trained language model like GPT-3 or BERT to generate summaries.

